In [40]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix


In [2]:
data = pd.read_csv('capstone/yagnesh/full_data.csv')

In [3]:
pd.set_option('max_columns', None)
# pd.reset_option(“max_columns”)

In [43]:
data.head()

,duration,end,end_idx,file,line,onset_count,onset_rate,onset_time_diff_mean,onset_time_diff_var,pitch_25pct,pitch_2pct,pitch_50pct,pitch_75pct,pitch_98pct,pitch_log_diff_variance,pitch_log_mean,pitch_log_stdev,speaker,speaker_role,start,start_idx,text,word_count,word_rate,BERT-GS_Scores,Grandstanding_ind
0,32.08,158.44,2535040,19-123.wav,2,85.0,2.649626,0.365333,0.111914,181.819070,152.614304,200.578947,223.845532,544.841599,0.002126,5.417890,0.353185,John_G_Roberts_Jr,scotus_justice,126.36,2021760,"Ms. Windham, this is a case involving free exe...",75,2.337905,-0.033068,1
1,23.32,213.40,3414400,19-123.wav,6,57.0,2.444254,0.396000,0.111454,168.666118,155.043824,179.213114,193.746492,463.258206,0.001241,5.239607,0.215422,John_G_Roberts_Jr,scotus_justice,190.08,3041280,-- you don't see any difference in terms of th...,57,2.444254,-0.722720,0
2,46.56,297.56,4760960,19-123.wav,15,106.0,2.276632,0.440686,0.141785,106.406876,91.573887,121.349670,864.882127,1073.811830,0.000718,5.475356,0.977699,Clarence_Thomas,scotus_justice,251.00,4016000,"Thank you, Mr. Chief Justice. Counsel, followi...",86,1.847079,-1.297273,0
3,70.44,443.44,7095040,19-123.wav,19,175.0,2.484384,0.401655,0.126817,167.211747,128.685006,191.521124,264.665508,936.866783,0.002082,5.390974,0.402863,Stephen_G_Breyer,scotus_justice,373.00,5968000,Yes. Thank you. On pages 45 and 46 of the City...,142,2.015900,0.130100,1
4,74.92,606.28,9700480,19-123.wav,21,193.0,2.576081,0.383333,0.139285,193.746492,135.425885,249.815142,318.399232,1399.987975,0.002184,5.630952,0.591756,Stephen_G_Breyer,scotus_justice,531.36,8501760,"Well, you don't have to say, according to them...",196,2.616124,0.978803,1


In [5]:
data['BERT-GS_Scores'].describe()

count    2545.000000
mean       -0.090471
std         0.674580
min        -1.887697
25%        -0.560338
50%        -0.066163
75%         0.396551
max         1.778646
Name: BERT-GS_Scores, dtype: float64

In [7]:
# data['Grandstanding_ind'] = np.where(data['bert-gs_scores']>=1,1,0)

data['Grandstanding_ind'] = np.where(data['BERT-GS_Scores']>data['BERT-GS_Scores'].mean(),1,0)


In [8]:
# data

In [9]:
data.describe()

,duration,end,end_idx,line,onset_count,onset_rate,onset_time_diff_mean,onset_time_diff_var,pitch_25pct,pitch_2pct,pitch_50pct,pitch_75pct,pitch_98pct,pitch_log_diff_variance,pitch_log_mean,pitch_log_stdev,start,start_idx,word_count,word_rate,BERT-GS_Scores,Grandstanding_ind
count,2545.000000,2545.000000,2.545000e+03,2545.000000,2493.000000,2493.000000,2493.000000,2493.000000,2468.000000,2468.000000,2468.000000,2468.000000,2468.000000,2468.000000,2468.000000,2468.000000,2545.000000,2.545000e+03,2545.000000,2545.000000,2545.000000,2545.000000
mean,35.888680,2497.763914,3.996422e+07,127.150098,107.832732,3.148346,0.335227,0.100422,204.017436,170.510557,233.717239,279.592222,472.098732,0.001650,5.392739,0.249686,2461.875234,3.939000e+07,91.086051,2.630631,-0.090471,0.514342
std,22.562956,1524.074632,2.438519e+07,85.015071,66.198166,0.896391,0.097852,0.070649,134.059419,112.075911,153.716852,179.402578,311.686606,0.000803,0.365028,0.178992,1523.170614,2.437073e+07,53.501933,1.216303,0.674580,0.499892
min,-1.000000,141.250000,2.260000e+06,2.000000,12.000000,0.500835,0.136238,0.009792,65.406391,65.406391,65.406391,65.785287,66.135890,0.000029,4.184086,0.004621,119.400000,1.910400e+06,40.000000,-52.000000,-1.887697,0.000000
25%,21.160000,1198.240000,1.917184e+07,58.000000,66.000000,2.512356,0.265877,0.054148,148.538494,125.700919,166.728822,189.321316,281.175501,0.001127,5.168921,0.141251,1163.800000,1.862080e+07,55.000000,2.257252,-0.560338,0.000000
50%,29.445000,2375.230000,3.800368e+07,117.000000,90.000000,3.038508,0.320727,0.081741,179.730700,152.829465,204.676132,248.372169,387.492984,0.001589,5.409643,0.199095,2343.120000,3.748992e+07,74.000000,2.594340,-0.066163,1.000000
75%,43.000000,3568.165000,5.709064e+07,184.000000,128.000000,3.674833,0.389486,0.127898,220.000000,182.151525,248.372169,288.619783,532.305673,0.002093,5.555637,0.296010,3536.345000,5.658152e+07,109.000000,2.965404,0.396551,1.000000
max,233.640000,6964.480000,1.114317e+08,472.000000,733.000000,6.873315,1.031680,0.833887,2063.023295,2024.997863,2080.949703,2080.949703,2093.004522,0.007706,7.634803,1.091742,6934.240000,1.109478e+08,510.000000,6.010929,1.778646,1.000000


In [10]:
# data

In [11]:
data.columns

Index(['duration', 'end', 'end_idx', 'file', 'line', 'onset_count',
       'onset_rate', 'onset_time_diff_mean', 'onset_time_diff_var',
       'pitch_25pct', 'pitch_2pct', 'pitch_50pct', 'pitch_75pct',
       'pitch_98pct', 'pitch_log_diff_variance', 'pitch_log_mean',
       'pitch_log_stdev', 'speaker', 'speaker_role', 'start', 'start_idx',
       'text', 'word_count', 'word_rate', 'BERT-GS_Scores',
       'Grandstanding_ind'],
      dtype='object')

In [30]:
#all features
new_data = data[['onset_count',
       'onset_rate', 'onset_time_diff_mean', 'onset_time_diff_var',
       'pitch_25pct', 'pitch_2pct', 'pitch_50pct', 'pitch_75pct',
       'pitch_98pct', 'pitch_log_diff_variance', 'pitch_log_mean',
       'pitch_log_stdev', 'word_count', 'word_rate', 'Grandstanding_ind']].copy()

In [31]:
# new_data

In [32]:
new_data = new_data.dropna()

In [33]:
new_data.head()

,onset_count,onset_rate,onset_time_diff_mean,onset_time_diff_var,pitch_25pct,pitch_2pct,pitch_50pct,pitch_75pct,pitch_98pct,pitch_log_diff_variance,pitch_log_mean,pitch_log_stdev,word_count,word_rate,Grandstanding_ind
0,85.0,2.649626,0.365333,0.111914,181.819070,152.614304,200.578947,223.845532,544.841599,0.002126,5.417890,0.353185,75,2.337905,1
1,57.0,2.444254,0.396000,0.111454,168.666118,155.043824,179.213114,193.746492,463.258206,0.001241,5.239607,0.215422,57,2.444254,0
2,106.0,2.276632,0.440686,0.141785,106.406876,91.573887,121.349670,864.882127,1073.811830,0.000718,5.475356,0.977699,86,1.847079,0
3,175.0,2.484384,0.401655,0.126817,167.211747,128.685006,191.521124,264.665508,936.866783,0.002082,5.390974,0.402863,142,2.015900,1
4,193.0,2.576081,0.383333,0.139285,193.746492,135.425885,249.815142,318.399232,1399.987975,0.002184,5.630952,0.591756,196,2.616124,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,96.0,2.941176,0.340211,0.082762,258.620539,237.513276,273.998917,304.020834,459.433274,0.002369,5.660273,0.163730,89,2.726716,0
2541,47.0,3.996599,0.238609,0.048783,264.665508,235.844798,285.304702,309.335024,361.543738,0.002402,5.658832,0.112642,41,3.486395,0
2542,75.0,3.465804,0.288000,0.071957,254.177593,227.286038,273.998917,307.553386,472.047384,0.002538,5.651103,0.165456,63,2.911275,0
2543,67.0,3.978622,0.242424,0.047602,254.177593,234.432110,283.661468,323.498278,379.914836,0.002915,5.666474,0.143716,53,3.147268,1


In [34]:
X = new_data.loc[:, new_data.columns != 'Grandstanding_ind']
y = new_data.loc[:, new_data.columns == 'Grandstanding_ind']

In [35]:
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
import statsmodels.api as sm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=117)



# logreg = LogisticRegression(max_iter=1000)
# # logreg.fit(os_data_X, os_data_y.values.ravel())
# logreg.fit(X_train, y_train.values.ravel())

model = sm.Logit(endog=y_train, exog=X_train).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.585279
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:      Grandstanding_ind   No. Observations:                 1974
Model:                          Logit   Df Residuals:                     1960
Method:                           MLE   Df Model:                           13
Date:                Sat, 13 Nov 2021   Pseudo R-squ.:                  0.1556
Time:                        23:21:53   Log-Likelihood:                -1155.3
converged:                       True   LL-Null:                       -1368.2
Covariance Type:            nonrobust   LLR p-value:                 8.425e-83
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
onset_count                -0.0049      0.004     -1.163      0.245      -0.013   

In [36]:
pred = model.predict(exog=X_test)
pred.head()

1777    0.608400
154     0.879193
2368    0.619884
345     0.180810
1611    0.552859
dtype: float64

In [41]:
confusion_matrix(y_true=y_test, y_pred=list(round(pred)))


array([[162,  63],
       [ 91, 178]])

In [42]:
accuracy_score(y_true=y_test, y_pred=list(round(pred)))


0.6882591093117408

In [44]:
#selected features
new_data2 = data[[
       'onset_rate', 'onset_time_diff_mean', 'onset_time_diff_var',
        'pitch_75pct',
        'pitch_log_diff_variance', 
       'pitch_log_stdev', 'word_count', 'word_rate', 'Grandstanding_ind']].copy()

In [45]:
new_data2 = new_data2.dropna()
X = new_data2.loc[:, new_data2.columns != 'Grandstanding_ind']
y = new_data2.loc[:, new_data2.columns == 'Grandstanding_ind']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=117)

In [48]:
model = sm.Logit(endog=y_train, exog=X_train).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.586228
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:      Grandstanding_ind   No. Observations:                 1974
Model:                          Logit   Df Residuals:                     1966
Method:                           MLE   Df Model:                            7
Date:                Sat, 13 Nov 2021   Pseudo R-squ.:                  0.1542
Time:                        23:34:51   Log-Likelihood:                -1157.2
converged:                       True   LL-Null:                       -1368.2
Covariance Type:            nonrobust   LLR p-value:                 4.745e-87
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
onset_rate                 -0.6811      0.069     -9.932      0.000      -0.816   

In [49]:
pred = model.predict(exog=X_test)
pred.head()

1777    0.604909
154     0.859670
2368    0.642046
345     0.183635
1611    0.559790
dtype: float64

In [50]:
confusion_matrix(y_true=y_test, y_pred=list(round(pred)))


array([[161,  64],
       [ 87, 182]])

In [51]:
accuracy_score(y_true=y_test, y_pred=list(round(pred)))


0.694331983805668

In [ ]:
newdf = data[data['speaker'] == 'Elena_Kagan']

In [75]:
#selected features
new_data3 = data[[
       'onset_rate', 'onset_time_diff_mean', 'onset_time_diff_var',
        'pitch_75pct',
        'pitch_log_diff_variance', 
       'pitch_log_stdev', 'word_count', 'word_rate', 'Grandstanding_ind', 'speaker']].copy()

In [76]:
new_data4 = new_data3[new_data3['speaker']== 'Elena_Kagan']

In [77]:
# new_data4 = new_data4.reset_index()

In [78]:
new_data4 = new_data4.drop(['speaker'], axis=1)

In [79]:
new_data4.head()

,onset_rate,onset_time_diff_mean,onset_time_diff_var,pitch_75pct,pitch_log_diff_variance,pitch_log_stdev,word_count,word_rate,Grandstanding_ind
12,3.308081,0.302031,0.074988,263.141147,0.001993,0.182775,97,2.449495,0
13,2.500000,0.374476,0.108417,273.998917,0.001210,0.264363,86,2.529412,1
14,3.340675,0.295467,0.060372,253.079627,0.001409,0.188590,73,2.679883,1
28,3.236493,0.302476,0.063352,267.740773,0.001159,0.174726,88,2.242610,0
29,3.236994,0.288582,0.041909,251.258115,0.001970,0.115236,56,3.236994,1


In [80]:
new_data4 = new_data4.dropna()
X = new_data4.loc[:, new_data4.columns != 'Grandstanding_ind']
y = new_data4.loc[:, new_data4.columns == 'Grandstanding_ind']

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=117)

In [82]:
model = sm.Logit(endog=y_train, exog=X_train).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.557718
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:      Grandstanding_ind   No. Observations:                  276
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                            7
Date:                Sat, 13 Nov 2021   Pseudo R-squ.:                  0.1741
Time:                        23:43:28   Log-Likelihood:                -153.93
converged:                       True   LL-Null:                       -186.38
Covariance Type:            nonrobust   LLR p-value:                 1.574e-11
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
onset_rate                 -1.6826      0.334     -5.038      0.000      -2.337   

In [83]:
pred = model.predict(exog=X_test)
pred.head()

1725    0.093301
1614    0.956076
768     0.581060
505     0.124645
1997    0.838273
dtype: float64

In [84]:
confusion_matrix(y_true=y_test, y_pred=list(round(pred)))


array([[16, 11],
       [17, 26]])

In [85]:
accuracy_score(y_true=y_test, y_pred=list(round(pred)))


0.6